# Preprocess subject data

In [1]:
import sys
import os
from pathlib import Path
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject, Group
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 


imported src module
imported preprocessing module
imported configs module
imported utils module


Load in subject IDs and paths from CFGLog

In [2]:
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC
Panc_subject_ids = config.data.chronic_pancreatitis.subject_ids.CP
Lupus_CP_subject_ids = config.data.lupus.subject_ids.CP
Lupus_NP_subject_ids = config.data.lupus.subject_ids.NP

In [3]:
times_tup, time_win_path = pre_utils.get_time_window(configs.PERISTIM_TIME_WIN)

AttributeError: module 'src.configs.config' has no attribute 'PERISTIM_TIME_WIN'

Combine subject IDs

In [ ]:
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids,
    "chronic_pancreatitis": Panc_subject_ids,
    "lupus": Lupus_CP_subject_ids + Lupus_NP_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain']) + len(sub_ids['chronic_pancreatitis']) + len(sub_ids['lupus'])}")
print(sub_ids)

Define preprocessing parameters

In [ ]:
TIME_RANGE = (-2.5, 2.5)
PERISTIM_TIME_WIN = abs(TIME_RANGE[0]) + abs(TIME_RANGE[1])
# BASELINE = (-0.2, 0)

Create Subject object for each subject

In [ ]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        print(subject.subject_folder)
        subject.preprocess()
        subject.get_cleaned_epochs(TIME_RANGE, PERISTIM_TIME_WIN)           
        break

        # subject.get_cleaned_resting()

        # subject.get_stc_resting()
        # subject.get_stc_epochs()


In [ ]:
subject.preprocessed_raw

In [ ]:
# Create an instance of SubjectProcessor
processor = SubjectProcessor(paths_dict, roi_acronyms)

# Loop through each subject and process the response
for sub_id in sub_ids_focus:
    subject = Subject(sub_id)
    processor.plot_TFR_and_trace(subject,
                                 channel=channel, 
                                 baseline=baseline,
                                 time_range=time_range,
                                 vlim=vlim,
                                 )
    # processor.process_response(subject)
    # break
    

# Display the results
processor.display_results()
